### 00 OpenAI API KEY 환경변수 등록

In [9]:
import os, json

with open('conf.json', 'r') as f:
        json_data = json.load(f)
    
os.environ['OPENAI_API_KEY'] = json_data['openai_config']["API_KEY"]

### 01 설정

In [10]:
class CONFIG:
    etf_tkr="AIQ"

In [11]:
from data.get_data import get_pdf_df

pdf_df = get_pdf_df(etf_tkr=CONFIG.etf_tkr)

tkr_list = pdf_df.child_stk_tkr.to_list()

### 02 영향도 상위 5개 기업 추출

#### 02-1 2분기 가격데이터 로드

In [12]:
from data.get_data import get_prices_df
from data.fetch_data import fetch_data_from_db

prices_df = get_prices_df(tickers=tkr_list, start_date="2023-03-01")

#### 02-2 유동주식수 로드

In [13]:
import pandas as pd

sql = """
    SELECT * FROM os_stk_info;
"""

stk_info_df = pd.DataFrame(fetch_data_from_db(query=sql)).dropna()

stk_info_df = stk_info_df[stk_info_df["stk_tkr"].isin(prices_df.columns.str.upper().to_list())].reset_index(drop=True)

stk_info_df.index = stk_info_df['stk_tkr'].str.lower()

stk_info_df = stk_info_df[['float_shares']]

#### 02-3 영향도 상위 5개 기업 산출

In [14]:
시작_영향도 = stk_info_df['float_shares'] * prices_df.iloc[0]

끝_영향도 = stk_info_df['float_shares'] * prices_df.iloc[-1]

영향도 = abs(끝_영향도 - 시작_영향도)

영향도_top5_종목들_리스트 = 영향도.sort_values(ascending=False).head().index.str.upper().to_list()

영향도_top5_종목들_리스트

['AAPL', 'MSFT', 'NVDA', 'GOOGL', 'AMZN']

### 03 뉴스데이터 로드

In [15]:
import yfinance as yf
from tqdm.auto import tqdm

stk_infos = yf.Tickers(영향도_top5_종목들_리스트)

display(tkr_list[0])

text = {
    f"{CONFIG.etf_tkr} news": []
}

for tkr in tqdm(영향도_top5_종목들_리스트):
    
    tmp = {}
    tmp["stock"] = tkr
    tmp["news_titles"] = []
    
    news_infos = stk_infos.tickers[tkr].news
    
    for news in news_infos:
        tmp['news_titles'].append(news["title"])
        
    text[f"{CONFIG.etf_tkr} news"].append(tmp)
    
text = str(text)

print(text)

'AAPL'

  0%|          | 0/5 [00:00<?, ?it/s]

{'AIQ news': [{'stock': 'AAPL', 'news_titles': ['Apple Set to Unveil Latest iPhone: 5 Things to Watch', 'Dow Jones Futures Fall: Tesla Stock Breaks Out; Key Inflation Reports, Apple iPhone 15 Event Next', 'Asian Equities Fall on Fragile China Sentiment: Markets Wrap', 'Qualcomm Inks Deal With Apple to Supply iPhone Chips Through 2026', 'Apple’s iPhone launch is in less than 24 hours. Here’s what to expect as CEO Tim Cook tries to reverse three painful quarters of sales shrinkage', 'Time to Buy Apple or Qualcomm Stock for Their Renewed Partnership', 'Apple launch event, Google vs. DOJ: What to watch', 'Google Goes to Federal Court. The Future of Internet Search Is at Stake.']}, {'stock': 'MSFT', 'news_titles': ['Microsoft (MSFT) Outpaces Stock Market Gains: What You Should Know', 'More writers sue OpenAI for copyright infringement over AI training', "ChatGPT's development nearly cost this small Midwestern city its drinking water", 'Top Stock Reports for Microsoft, Novo Nordisk & Booking

### 03 프롬프트 엔지니어링

In [16]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessage, HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI


template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                f"""
                Summarize the common issues based on the news headlines of stocks which are best 5 influential components of {CONFIG.etf_tkr} in Korean.
                
                - Tone : Humble
                - Style : MECE, accurate
                - Reader level : college student
                - Length : within 500 characters
                - Perspective : Mckinsey&Company senior consultant
                - Format : markdown
                
                You MUST answer in Korean. 
                """
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

llm = ChatOpenAI(model="gpt-3.5-turbo-16k")

answer = llm(template.format_messages(text=text)).content

print(answer)

AIQ에서 가장 영향력 있는 구성 요소인 5개 주식에 관한 뉴스 헤드라인을 기반으로 일반적인 문제들을 요약하면 다음과 같습니다.

1. Apple (AAPL):
   - 최신 iPhone 공개를 앞두고 관심을 받고 있음
   - 중국 심리에 민감하여 아시아 지수 하락
   - Qualcomm과의 파트너십 갱신으로 인한 주식 매수 기회 제시

2. Microsoft (MSFT):
   - 주식 시장 성장률을 앞섬
   - OpenAI에 대한 저작권 침해 소송 발생
   - AI 개발로 인한 문제로 인해 도시의 음수 수돗물 위기

3. NVIDIA (NVDA):
   - AI 모델 실행 속도에 대한 새로운 벤치마크 테스트 결과 발표
   - 주식 시장 압력에도 불구하고 NVDA 주식은 강하게 유지됨
   - 장기적인 성장이 예상되는 AI 산업에 초점을 맞춘 ETF

4. Alphabet (GOOGL/GOOG):
   - Google의 대규모 반독점 소송에 대한 관심
   - 주식 시장에 비해 시장 점유율이 낮음
   - Apple의 이벤트 및 DOJ 소송에 주목

5. Amazon (AMZN):
   - 시장의 실적을 이끄는 기업 중 하나로 주목
   - 주식의 가격 변동을 제어할 수 없지만 대응 방안에 대해서는 제어 가능
   - 중국의 새로운 기업 Temu가 Amazon을 견제할 가능성

이러한 주요 이슈들을 주목하며 AIQ의 영향력 있는 주식들에 대한 투자 결정을 내리는 것이 중요합니다.
